# 随机森林(RF)案例

### 案例：使用sklearn构建RF模型，预测学生最终是否通过考试，对模型进行调优
- sklearn实现决RF
- 可视化RF

### 数据集：student_data_RF.csv
该数据集取自一所学校K12阶段学生的在线行为日志数据。具体字段如下：
- content，学生查看课程资料的次数
- assessment，学生参加测验的次数
- check_grade，查看成绩次数
- interaction，根据日志中的时间和位置信息计算出的学生类别，共7类
- passCourse，是否通过期末考试（0-YES，1-NO）

## 1.读取数据
pandas读取数据

In [1]:
import pandas as pd
data = pd.read_csv('student_data_RF.csv', index_col=0) #使用pandas读取数据

## 2.模型分析

### 2.1 划分数据集
- sklearn.model_selection 的 train_test_split方法

In [2]:
from sklearn.model_selection import train_test_split

y = data['isPassExam'].values #获取目标变量（label）
X = data.drop(['isPassExam'], axis=1).values#丢弃目标变量，得到输入变量
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)#划分测试集和训练集

print('train dataset: {0}; test dataset: {1}'.format(
    X_train.shape, X_test.shape)) #训练集和测试集的样本数

train dataset: (1285, 4); test dataset: (322, 4)


###  2.2 模型构建
构建随机森林模型，使用sklearn.ensemble的RandomForestClassifier()方法，参数如下：

- n_estimators:森林里（决策）树的数目.整数，可选择(默认值为10)。
- criterion:特征选择算法，可选择信息熵(entropy)或基尼不纯度(gini);
- max_depth:指定树的最大深度，可解决过拟合问题；
- min_samples_split:内部节点再划分所需最小样本数，小于此值不创建分支，一般是2，是一种前剪枝办法；适用于样本量大的情况；
- min_samples_leaf:叶子节点需要的最小的样本个数，如果叶子节点数小于设定的值，则会跟兄弟节点一起被剪枝，可以设定具体个数或者百分比，这也是一种前剪枝办法；
- max_leaf_nodes:限制最大的叶子节点个数，即叶子个数最多不能超过的值，主要是防止过拟合
- min_impurity_decrease：最小不纯度，这个值限制了决策树的增长，如果某节点的不纯度(基尼系数，信息增益，均方差，绝对差)小于这个阈值，则该节点不再生成子节点,即为叶子节点;
- max_features：划分时考虑的最大特征数，如果是None表示考虑所有特征，也可以是其它指定的个数


In [3]:
from sklearn.ensemble import RandomForestClassifier #使用这个包构建随机森林

rf = RandomForestClassifier(n_estimators=5) #构建随机森林

###  2.3 训练和评估模型
- 训练模型RandomForestClassifier().fit()，填入训练数据的x和y
- 评估模型score()获得准确率，sklearn.metrics 的 precision_recall_curve方法获得精度 召回率等

In [4]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

rf.fit(X_train, y_train)#拟合模型

train_score = rf.score(X_train, y_train)
test_score = rf.score(X_test, y_test)

print('train_score',train_score)

print('test_score',test_score)



train_score 0.978988326848249
test_score 0.8509316770186336


### 2.3 交叉验证
- sklearn.model_selection的cross_val_score方法

In [5]:
from sklearn.model_selection import  cross_val_score

print('X.shape',X.shape)
print('y.shape',y.shape)
scores = cross_val_score(rf, X, y, cv=5) #五次交叉验证结果
print('scores',scores)
scores_series = pd.Series(scores)
scores_mean = scores_series.mean() #求得五次交叉验证结果平均值
print('scores_mean',scores_mean)

X.shape (1607, 4)
y.shape (1607,)
scores [0.83229814 0.83540373 0.85981308 0.84735202 0.88161994]
scores_mean 0.8512973820166019


### 2.4 随机森林可视化
- 只能用rf.estimators_[]一次导出其中一棵树
- 先安装graphviz工具，将其安装路径加入环境变量
- 使用sklearn.tree.export_graphviz()方法画决策树，生成可视化文件

In [6]:
from sklearn.tree import export_graphviz
from sklearn import tree
import pydotplus

 
estimator = rf.estimators_[0] #获取其中一棵树
feature_name=['content','assessment','check_grade','interaction']
dot_data = tree.export_graphviz(estimator,feature_names=feature_name,out_file=None)
graph = pydotplus.graph_from_dot_data(dot_data)
graph.write_pdf("students_rf.pdf")

# 1. 在电脑上安装 graphviz，将其加入环境变量
# 2. 运行 `dot -Tpng titanic.dot -o titanic.png` 
# 3. 在当前目录查看生成的决策树 titanic.png

True

## 3.与决策树对比

In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier #使用这个包构建决策树
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import  cross_val_score

import pandas as pd

#读取数据
data = pd.read_csv('student_data_RF.csv', index_col=0) #使用pandas读取数据
 
y = data['isPassExam'].values #获取目标变量（label）
X = data.drop(['isPassExam'], axis=1).values#丢弃目标变量，得到输入变量

#划分数据及
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)#划分测试集和训练集

print('train dataset: {0}; test dataset: {1}'.format(
    X_train.shape, X_test.shape)) #训练集和测试集的样本数

#构建和训练模型
clf_dt = DecisionTreeClassifier() #构建决策树

clf_dt.fit(X_train, y_train)#拟合模型

train_predict = clf_dt.predict(X_train)
test_predict = clf_dt.predict(X_test)

#交叉验证
scores = cross_val_score(clf_dt, X, y, cv=5) #五次交叉验证结果
print('scores',scores)
scores_series = pd.Series(scores)
scores_mean = scores_series.mean() #求得五次交叉验证结果平均值
print('scores_mean',scores_mean)

#决策树可视化
import pydotplus  
#导出为pdf
feature_name=['content','assessment','check_grade','interaction']
dot_data = tree.export_graphviz(clf_dt,feature_names=feature_name, out_file=None)
graph = pydotplus.graph_from_dot_data(dot_data)
graph.write_pdf("students_dt.pdf")


train dataset: (1285, 4); test dataset: (322, 4)
scores [0.79813665 0.79192547 0.79127726 0.80996885 0.80685358]
scores_mean 0.7996323600549525


True

## 4.案例练习
重复上述案例，设置不同的随机森林参数，使得结果尽可能高；
